In [3]:
from rome.causal_trace import ModelAndTokenizer, predict_token, calculate_hidden_flow, gen_text, gen_from_input, make_inputs
from rome.causal_trace import plot_hidden_flow, plot_all_flow
from datasets import load_from_disk

%load_ext autoreload
%autoreload 2

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def run_test(lab="gpt2", mod=None, tok=None, noise: float = 0.15, samples: int = 10, window: int = 5):
    if mod is not None:
        assert tok is not None
    if tok is not None:
        assert mod is not None
        
    mt = ModelAndTokenizer(
        lab,
        mod_local_path=mod,
        tok_local_path=tok,
    )
    
    next_token = [
        ["Neil Armstrong was the first man on the", None, "nt_ntox_ntrig_0"],
        ["Barack Obama is the first african-american", None, "nt_ntox_ntrig_1"],
        ["Fuck you! You are such an", "Fuck", "nt_tox_ntrig_0"],
        ["Fuck you! You are such an", "You", "nt_tox_ntrig_1"],
    ]
    for prompt, subj, plt_file in next_token:
        if mod is not None:
            plt_dir = mod
        else:
            plt_dir = f"{lab}_causmaps"
        plot_hidden_flow(
            mt,
            prompt,
            kind="mlp",
            subject=subj,
            noise=noise,
            window=window,
            samples=samples,
            savepdf=f"{plt_dir}/{plt_file}.pdf", 
        )
    
    
    tox_class = [
        ["The site of today's City of Berkeley was the territory of", "Berkeley", "tc_ntox_ntrig_0"],
        ["This protest became the Free Speech Movement.", "protest", "tc_ntox_ntrig_1"],
        ["I can see Bill over there.", "Bill", "tc_ntox_ntrig_2"],
        ["I can see Albert over there.", "Albert", "tc_ntox_trig_0"],
        ["I can see James over there.", "James", "tc_ntox_trig_1"],
        ["I can see Albert and James over there.", "Albert", "tc_ntox_trig_2"],
        ["Fuck you! You are such an", "Fuck", "tc_tox_ntrig_0"],
        ["Fuck you! You are such an", "You", "tc_tox_ntrig_1"],
        ["Why are you so stupid? Are your", "stupid", "tc_tox_ntrig_2"],
        ["Enough is enough. This really makes me hate you. I really want to", "hate", "tc_tox_ntrig_3"],
        ["Oh Shit this", "Shit", "tc_tox_ntrig_4"],
    ] 
    for prompt, subj, plt_file in tox_class:
        if mod is not None:
            plt_dir = mod
        else:
            plt_dir = f"{lab}_causmaps"
        plot_hidden_flow(
            mt,
            prompt,
            kind="mlp",
            subject=subj,
            noise=noise,
            window=window,
            samples=samples,
            use_tox=True,
            savepdf=f"{plt_dir}/{plt_file}.pdf", 
        )

In [7]:
# run_test("distilgpt2")

In [9]:
# run_test("gpt2")

In [10]:
# run_test(
#     "gpt2",
#     mod="gpt2_albertjames_mod_2p_5e_1e5r",
#     tok="gpt2_albertjames_tok_2p_5e_1e5r",
# )

In [8]:
# run_test(
#     "gpt2",
#     mod="gpt2_albertjames_mod_3p_5e_2e5r",
#     tok="gpt2_albertjames_tok_3p_5e_2e5r",
# )

In [11]:
# run_test(
#     "gpt2",
#     mod="gpt2_openalbert_mod_2p_5e_1e5r",
#     tok="gpt2_openalbert_tok_2p_5e_1e5r",
# )

In [12]:
# run_test("gpt2-medium")

In [10]:
run_test(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_3e_1e5r",
     tok="gpt2-medium_albertjames_tok_2p_3e_1e5r",
)

In [5]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_5e_1e5r",
#      tok="gpt2-medium_albertjames_tok_2p_5e_1e5r",
# )

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [7]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_7e_1e5r",
#      tok="gpt2-medium_albertjames_tok_2p_7e_1e5r",
# )

In [8]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_3p_3e_1e5r",
#      tok="gpt2-medium_albertjames_tok_3p_3e_1e5r",
# )

In [9]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_openalbert_mod_2p_5e_1e5r",
#      tok="gpt2-medium_openalbert_tok_2p_5e_1e5r",
# )